In [23]:
# 共通で利用するライブラリ
import glob
import os
import datetime
import pickle
from natsort import natsorted
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline
%load_ext autoreload

# 追加するライブラリ
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Ch.09 小規模機械学習システムを作成する

- 月次で更新するタイミングで、データを蓄積する
- 蓄積されたデータをもとに毎月のモデル構築を行う
- データ一覧
  - m_area.csv
  - m_store.csv
  - tbl_order_202004.csv ~ tbl_order_202008.csv
  - store_monthly_data.csv
  - ml_base_data.csv
  - model_y_weekday_GradientBoosting.pickle
  - model_y_weekend_GradientBoosting.pickle
  - X_cols.csv

- フォルダ構成

```bash
ch09
├── ch09.ipynb
├── models
│   ├── model_y_weekday_GradientBoosting.pickle
│   ├── model_y_weekend_GradientBoosting.pickle
│   └── X_cols.csv
└── data
    ├── 00_input
    │   └── tbl_order_YYYYMM.csv
    ├── 01_store_monthly
    │   └── store_monthly_data.csv
    ├── 02_ml_base
    │   └── ml_base_data.csv
    ├── 10_output_ml_result
    │   └── results_xxxx
    ├── 11_output_report
    └── 99_master
```

### Knock81: フォルダを生成して初期の変数定義を行う

In [2]:
# フォルダ作成

data_dir = "data"

input_dir = os.path.join(data_dir, "00_input")
store_monthly_dir = os.path.join(data_dir, "01_store_monthly")
ml_base_dir = os.path.join(data_dir, "02_ml_base")

output_ml_result_dir = os.path.join(data_dir, "10_output_ml_result")
output_report_dir = os.path.join(data_dir, "11_output_report")

master_dir = os.path.join(data_dir, "99_master")
model_dir = "models"

os.makedirs(input_dir, exist_ok=True)
os.makedirs(store_monthly_dir, exist_ok=True)
os.makedirs(ml_base_dir, exist_ok=True)
os.makedirs(output_ml_result_dir, exist_ok=True)
os.makedirs(output_report_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [3]:
# 初期変数の定義

tg_ym = "202004"

target_file = "tbl_order_" + tg_ym + ".csv"
m_area_file = "m_area.csv"
m_store_file = "m_store.csv"
store_monthly_file = "store_monthly_data.csv"
ml_base_file = "ml_base_data.csv"


### Knock82: 更新データを読み込んで店舗別データを作成する

In [4]:
# 更新データの読み込み

m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
target_data = pd.read_csv(os.path.join(input_dir, target_file))

# データの年月チェック
max_date = pd.to_datetime(target_data["order_accept_date"]).max()
min_date = pd.to_datetime(target_data["order_accept_date"]).min()
max_str_date = max_date.strftime("%Y%m")
min_str_date = min_date.strftime("%Y%m")
if (tg_ym == min_str_date) and (tg_ym == max_str_date):
    print("年月が一致しました。")
else:
    raise Exception("年月が一致しません。")

年月が一致しました。


In [5]:
# 店舗別集計を行うための関数

def calc_delta(t):
    """
    経過時間を計算する
    戻り値は単位（分）
    """
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60

# 1か月分データから店舗別に集計する


def data_processing(order_data, m_area, m_store) :
    """
    データの加工関数
    店舗ごとに集計する
    """

    # 店舗No.999の除外
    order_data = order_data.loc[order_data["store_id"] != 999]

    # オーダーデータにマスタデータを結合
    order_data = pd.merge(order_data, m_store, on="store_id", how="left")
    order_data = pd.merge(order_data, m_area, on="area_cd", how="left")

    # テイクアウト名の追加
    order_data.loc[order_data["takeout_flag"] == 0, "takeout_name"] = "デリバリー"
    order_data.loc[order_data["takeout_flag"] == 1, "takeout_name"] = "お持ち帰り"

    # ステイタス名の追加
    order_data.loc[order_data["status"] == 0, "status_name"] = "受付"
    order_data.loc[order_data["status"] == 1, "status_name"] = "お支払済"
    order_data.loc[order_data["status"] == 2, "status_name"] = "お渡し済"
    order_data.loc[order_data["status"] == 9, "status_name"] = "キャンセル"

    # ピザ提供までの時間
    order_data.loc[:, "order_accept_datetime"] = pd.to_datetime(order_data["order_accept_date"])
    order_data.loc[:, "delivered_datetime"] = pd.to_datetime(order_data["delivered_date"])
    order_data.loc[:, "delta"] = order_data[[
        "order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)

    # ピザが注文された時間帯を追加
    order_data.loc[:, "order_accept_hour"] = order_data["order_accept_datetime"].dt.hour
    order_data.loc[:, "order_accept_weekday"] = order_data["order_accept_datetime"].dt.weekday
    # 平日/休日を追加
    order_data.loc[order_data["order_accept_weekday"] < 5, "weekday_info"] = "平日"
    order_data.loc[order_data["order_accept_weekday"] >= 5, "weekday_info"] = "休日"

    # 店舗単位の集計
    # 総オーダー数
    store_data = order_data.groupby(["store_name"]).count()[["order_id"]]
    # 完了しているオーダー数
    store_f = order_data.loc[
        (order_data["status_name"] == "お渡し済") | (order_data["status_name"] == "お支払済")
    ].groupby(["store_name"]).count()[["order_id"]]
    # キャンセル数
    store_c = order_data.loc[
        order_data["status_name"] == "キャンセル"
    ].groupby(["store_name"]).count()[["order_id"]]
    # デリバリー数
    store_d = order_data.loc[
        order_data["takeout_name"] == "デリバリー"
    ].groupby(["store_name"]).count()[["order_id"]]
    # お持ち帰り数
    store_t = order_data.loc[
        order_data["takeout_name"] == "お持ち帰り"
    ].groupby(["store_name"]).count()[["order_id"]]
    # 平日オーダー数
    store_weekday = order_data.loc[
        order_data["weekday_info"] == "平日"
    ].groupby(["store_name"]).count()[["order_id"]]
    # 休日オーダー数
    store_weekend = order_data.loc[
        order_data["weekday_info"] == "休日"
    ].groupby(["store_name"]).count()[["order_id"]]

    # 時間帯別オーダー数の集計
    times = order_data["order_accept_hour"].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[
            order_data["order_accept_hour"] == time
        ].groupby(["store_name"]).count()[["order_id"]]
        time_tmp.columns = [f"order_time_{time}"]
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis=1)

    # ピザ提供までの時間の平均時間
    store_delta = order_data.loc[
        order_data["status_name"] != "キャンセル"
    ].groupby(["store_name"]).mean()[["delta"]]

    # 各データの項目設定
    store_data.columns = ["order"]
    store_f.columns = ["order_fin"]
    store_c.columns = ["order_cancel"]
    store_d.columns = ["order_delivery"]
    store_t.columns = ["order_takeout"]
    store_weekday.columns = ["order_weekday"]
    store_weekend.columns = ["order_weekend"]
    store_delta.columns = ["delta_avg"]

    # 各データを結合
    store_data = pd.concat([
        store_data, store_f, store_c, store_d, store_t,
        store_weekday, store_weekend, store_time, store_delta
    ], axis=1)

    return store_data


In [11]:
# 店舗別集計データの作成

store_data = data_processing(target_data, m_area, m_store)
store_data.reset_index(drop=False, inplace=True)
store_data.loc[:, "year_month"] = tg_ym

display(store_data.head(1))

,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1147,945,202,848,299,842,305,92,125,112,101,97,105,103,97,110,109,96,34.5206,202004


### Knock83: 月次店舗データを更新する

In [13]:
# 店舗別データの追加更新

store_monthly_data = pd.read_csv(os.path.join(store_monthly_dir, store_monthly_file))
print(f"更新前: {len(store_monthly_data)}件")

store_monthly_data = pd.concat(
    [store_monthly_data, store_data], ignore_index=True, axis=0
)
store_monthly_data.loc[:, "year_month"] = store_monthly_data["year_month"].astype(str)

# 重複データの削除
store_monthly_data.drop_duplicates(
    subset=["store_name", "year_month"], inplace=True, keep="last"
)
print(f"更新後: {len(store_monthly_data)}件")

# データの出力
store_monthly_data.to_csv(os.path.join(store_monthly_dir, store_monthly_file), index=False)


更新前: 2535件
更新後: 2535件


### Knock84: 機械学習用データの作成と更新を行う

In [20]:
# 機械学習用データの作成

y = store_monthly_data[[
    "store_name", "year_month", "order_weekday", "order_weekend"
]].copy()

y.loc[:, "one_month_ago"] = pd.to_datetime(y["year_month"], format="%Y%m")
y.loc[:, "one_month_ago"] = y["one_month_ago"].map(
    lambda x: x - relativedelta(months=1)
)
y.loc[:, "one_month_ago"] = y["one_month_ago"].dt.strftime("%Y%m")

y_one_month_ago = y.copy()
y_one_month_ago.rename(columns={
    "order_weekday": "order_weekday_one_month_ago",
    "order_weekend": "order_weekend_one_month_ago",
    "year_month": "year_month_for_join"
}, inplace=True)

y = pd.merge(
    y, 
    y_one_month_ago[[
        "store_name", "year_month_for_join", 
        "order_weekday_one_month_ago", "order_weekend_one_month_ago"
    ]],
    left_on=["store_name", "one_month_ago"],
    right_on=["store_name", "year_month_for_join"],
    how="left"
)

y.dropna(inplace=True)
y.loc[
    y["order_weekday"] - y["order_weekday_one_month_ago"] > 0,
    "y_weekday"
] = 1
y.loc[
    y["order_weekday"] - y["order_weekday_one_month_ago"] <= 0,
    "y_weekday"
] = 0
y.loc[
    y["order_weekend"] - y["order_weekend_one_month_ago"] > 0,
    "y_weekend"
] = 1
y.loc[
    y["order_weekend"] - y["order_weekend_one_month_ago"] <= 0,
    "y_weekend"
] = 0

y.rename(columns={"year_month": "target_year_month"}, inplace=True)
y = y[[
    "store_name", "target_year_month", "one_month_ago",
    "y_weekday", "y_weekend"
]].copy()

ml_data = pd.merge(
    y, store_monthly_data,
    left_on=["store_name", "one_month_ago"],
    right_on=["store_name", "year_month"],
    how="left"
)
del ml_data["target_year_month"]
del ml_data["one_month_ago"]

display(ml_data.head(3))


,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0000,0.0000,1147,945,202,841,306,844,303,91,122,112,101,95,107,106,100,108,109,96,34.1101,201904
1,さいたま南店,1.0000,1.0000,1504,1217,287,1105,399,1104,400,130,135,147,143,142,137,130,113,140,132,155,35.3377,201904
2,さいたま緑店,1.0000,1.0000,1028,847,181,756,272,756,272,95,91,106,95,102,82,90,93,95,95,84,34.2916,201904


In [22]:
# 機械学習用データの更新

ml_base_data = pd.read_csv(os.path.join(ml_base_dir, ml_base_file))
print(f"更新前: {len(ml_base_data)}件")

ml_base_data = pd.concat([ml_base_data, ml_data], ignore_index=True)
ml_base_data.loc[:, "year_month"] = ml_base_data["year_month"].astype(str)
ml_base_data.drop_duplicates(
    subset=["store_name", "year_month"], inplace=True, keep="last"
)
print(f"更新後: {len(ml_base_data)}件")

ml_base_data.to_csv(os.path.join(ml_base_dir, ml_base_file), index=False)


更新前: 2340件
更新後: 2340件


### Knock85: 機械学習モデル用の事前データ加工を行う

In [24]:
# 機械学習用の事前データ加工

# One-Hotエンコーディング
category_data = pd.get_dummies(
    ml_base_data["store_name"], prefix="store", prefix_sep="_"
)
del category_data["store_麻生店"]
del ml_base_data["year_month"]
del ml_base_data["store_name"]
ml_base_data = pd.concat([ml_base_data, category_data], axis=1)

# データ分割
train_data, test_data = train_test_split(
    ml_base_data, test_size=0.3, random_state=0
)

print(f"Train: {len(train_data)}件 / Test: {len(test_data)}件")

print(f"Weekday Train_0: {len(train_data.loc[train_data['y_weekday'] == 0])}件")
print(f"Weekday Train_1: {len(train_data.loc[train_data['y_weekday'] == 1])}件")
print(f"Weekday Test_0: {len(test_data.loc[test_data['y_weekday'] == 0])}件")
print(f"Weekday Test_1: {len(test_data.loc[test_data['y_weekday'] == 1])}件")

print(f"Weekend Train_0: {len(train_data.loc[train_data['y_weekend'] == 0])}件")
print(f"Weekend Train_1: {len(train_data.loc[train_data['y_weekend'] == 1])}件")
print(f"Weekend Test_0: {len(test_data.loc[test_data['y_weekend'] == 0])}件")
print(f"Weekend Test_1: {len(test_data.loc[test_data['y_weekend'] == 1])}件")

Train: 1638件 / Test: 702件
Weekday Train_0: 777件
Weekday Train_1: 861件
Weekday Test_0: 311件
Weekday Test_1: 391件
Weekend Train_0: 843件
Weekend Train_1: 795件
Weekend Test_0: 355件
Weekend Test_1: 347件
